In [ ]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine

print("Pandas version: ", pd.__version__)
print("SQLAlchemy version: ", sqlalchemy.__version__)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# Postgres username, password, and database name
POSTGRES_ADDRESS = 'localhost' ## INSERT YOUR DB ADDRESS
POSTGRES_PORT = 5432 ## INSERT YOUR PORT (DEFAULT IS 5432)
POSTGRES_USERNAME = '' ## INSERT YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = '' ## INSERT YOUR POSTGRES PASSWORD
POSTGRES_DBNAME =  '' ## INSERT YOUR POSTGRES DATABASE NAME

# A long string that contains the necessary Postgres login information
# Create the connection
engine   = create_engine('postgresql+psycopg2://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(
    username=POSTGRES_USERNAME,
    password=POSTGRES_PASSWORD,
    ipaddress=POSTGRES_ADDRESS,
    port=POSTGRES_PORT,
    dbname=POSTGRES_DBNAME), pool_recycle=3600);

if engine:
    print("Connection successful!")
else:
    print("Connection unsuccessful.")

In [ ]:
dbConnection = engine.connect()

if dbConnection:
    print("Successfully connected to the database!")
else:
    print("Failed to connect to the database.")

In [ ]:
sofa_dem = pd.read_sql("""
    SELECT icu_adm.subject_id AS subject_id, icu_adm.hadm_id AS hadm_id, mimiciv_derived.sofa.stay_id AS stay_id,
    admissions.admittime AS admittime, admissions.dischtime AS dischtime, admissions.deathtime AS deathtime, admissions.hospital_expire_flag AS hospital_expire_flag,
    admissions.hospital_expire_flag AS hospital_expire_flag, admissions.insurance AS insurance, admissions.marital_status AS marital_status,  admissions.race AS race,
    patients.dod AS dod,starttime,sepsis3.suspected_infection_time, sepsis3.sofa_time
    FROM mimiciv_derived.sofa
    INNER JOIN mimiciv_icu.icustays AS icu_adm
    ON icu_adm.stay_id = mimiciv_derived.sofa.stay_id
    INNER JOIN mimiciv_hosp.admissions AS admissions
    ON admissions.hadm_id = icu_adm.hadm_id
    INNER JOIN mimiciv_hosp.patients AS patients
    ON patients.subject_id = icu_adm.subject_id
    INNER JOIN mimiciv_derived.sepsis3 AS sepsis3
    ON sepsis3.stay_id = mimiciv_derived.sofa.stay_id
    """, dbConnection);
sofa_dem.sort_values(['stay_id','starttime'])
sofa_dem.drop_duplicates(subset=['stay_id'],inplace=True)
# endtime is latest time at which the SOFA score is valid, endtime AS sofa_time
#sofa_dem

In [ ]:
sofa_dem_2 = pd.read_sql("""
    SELECT icu_adm.subject_id AS subject_id, icu_adm.hadm_id AS hadm_id, icu_adm.stay_id AS stay_id,
    icustay_detail.gender, icustay_detail.admission_age, height_1.height, weight.weight, weight.starttime AS we_time,
    charlson_comorbidity_index
    FROM mimiciv_derived.sepsis3 AS sepsis3
    INNER JOIN mimiciv_icu.icustays AS icu_adm
    ON icu_adm.stay_id = sepsis3.stay_id
    INNER JOIN mimiciv_derived.icustay_detail AS icustay_detail
    ON icustay_detail.stay_id = sepsis3.stay_id
    LEFT JOIN mimiciv_derived.height AS height_1
    ON height_1.stay_id = sepsis3.stay_id
    LEFT JOIN mimiciv_derived.weight_durations AS weight
    ON weight.stay_id = sepsis3.stay_id
    LEFT JOIN mimiciv_derived.charlson AS charlson
    ON charlson.hadm_id = icu_adm.hadm_id
    """, dbConnection);

sofa_dem_2.sort_values(['stay_id','we_time'])
sofa_dem_2.drop_duplicates(subset=['stay_id'],inplace=True)
sofa_dem_2.drop(labels = ['we_time','hadm_id','subject_id'], axis =1, inplace=True)

sofa_dem = sofa_dem.merge(sofa_dem_2, on=['stay_id'])
#sofa_dem

In [ ]:
sofa_val = pd.read_sql("""
    SELECT mimiciv_derived.sofa.stay_id AS stay_id, starttime,
    DATE(mimiciv_derived.sofa.starttime) AS date_now, respiration_24hours, coagulation_24hours, 
    liver_24hours, cardiovascular_24hours,cns_24hours, renal_24hours, sofa_24hours, sepsis3.suspected_infection_time
    FROM mimiciv_derived.sofa
    INNER JOIN mimiciv_derived.sepsis3 AS sepsis3
    ON sepsis3.stay_id = mimiciv_derived.sofa.stay_id
    """, dbConnection);
#print(f"Shape of the sepsis death dataset is: {sofa_val.shape}")
#print(sofa_val.head())
dbConnection.close();

In [ ]:
# Note surivaval now occus relative to when an infection is first identified
sofa_dem['survival']=pd.to_datetime(sofa_dem['dod'])-sofa_dem['suspected_infection_time']
sofa_dem['survival'] = sofa_dem['survival'].dt.days
sofa_dem.loc[sofa_dem['survival'] <=30, '30d_mortality_status'] = '1'
sofa_dem.loc[sofa_dem['survival'] > 30, '30d_mortality_status'] = '0' 
sofa_dem.loc[sofa_dem['survival'] <=90, '90d_mortality_status'] = '1'
sofa_dem.loc[sofa_dem['survival'] > 90, '90d_mortality_status'] = '0' 
sofa_dem['30d_mortality_status'] = sofa_dem['30d_mortality_status'].fillna(0)
sofa_dem['90d_mortality_status'] = sofa_dem['90d_mortality_status'].fillna(0)
#sofa_dem['time_infect']=sofa_dem['suspected_infection_time']-sofa_dem['starttime']
#sepsis3.suspected_infection_time, sepsis3.sofa_time

In [ ]:
sofa_val.sort_values(['stay_id','starttime'])
sofa_val.drop_duplicates(subset=['stay_id','date_now',
                               'respiration_24hours','coagulation_24hours',
                               'liver_24hours','cardiovascular_24hours',
                               'cns_24hours','renal_24hours','sofa_24hours'] ,inplace=True)
# remove datapoint that occured to prior to indentification of infection
sofa_val.drop(sofa_val[sofa_val['suspected_infection_time'].dt.floor('H') > sofa_val['starttime']].index,
                      inplace=True)

sofa_val = sofa_val.join(sofa_val.groupby(['stay_id'])['date_now'].min(), on='stay_id',how='left', rsuffix='_min')
sofa_val['day_count'] =(sofa_val['date_now']-sofa_val['date_now_min'])
sofa_val.drop(labels = 'date_now_min', axis =1, inplace=True)
sofa_val['day_count'] = pd.to_timedelta(sofa_val['day_count']).dt.days+1
day_max = 14
# Filter to keep only the first 14 days
sofa_val = sofa_val[sofa_val['day_count'] <= day_max]

# Take the maximum daily value for each stay_id and day_count
max_values = sofa_val.groupby(['stay_id', 'day_count']).agg({
    'respiration_24hours': 'max', 
    'coagulation_24hours': 'max', 
    'liver_24hours': 'max', 
    'cardiovascular_24hours': 'max',
    'cns_24hours': 'max', 
    'renal_24hours': 'max'
}).reset_index()                 

In [ ]:
# Create the pivot tables
df_max_daily = pd.DataFrame({'stay_id': max_values['stay_id'].unique()})
for column in ['respiration_24hours', 'coagulation_24hours', 'liver_24hours', 'cardiovascular_24hours', 'cns_24hours', 'renal_24hours']:
    df_pivot = max_values.pivot(index='stay_id', columns='day_count', values=column)
    df_pivot.columns = [f'{column}_d{day}' for day in df_pivot.columns]
    df_max_daily = df_max_daily.merge(df_pivot.reset_index(), on='stay_id')

# Merge the two DataFrames on 'stay_id'
df_max_daily = sofa_dem[['subject_id','hadm_id','stay_id', 
                         'insurance', 'marital_status', 'race',
                         'gender','admission_age','height','weight','charlson_comorbidity_index',
                         'survival', '30d_mortality_status', '90d_mortality_status']].merge(df_max_daily, on='stay_id', how='left')

# Define the mapping of old column names to new column names
column_mapping = {old: old.replace('_24hours', '_SOFA') for old in df_max_daily.columns if '_24hours' in old}

# Rename the columns
df_max_daily = df_max_daily.rename(columns=column_mapping)

c_list = df_max_daily.columns
for day_num in range(1,day_max+1):
    df_max_daily['SOFA_d{}'.format(day_num)] = df_max_daily[c_list[c_list.str.endswith('_d{}'.format(day_num))]].sum(axis=1, skipna=False)
   
for day_num in range(1,day_max):
    df_max_daily['delta{}_{}'.format(day_num,day_num+1)] = df_max_daily['SOFA_d{}'.format(day_num+1)]-df_max_daily['SOFA_d{}'.format(day_num)]
    df_max_daily['Area{}_{}'.format(day_num,day_num+1)] = (df_max_daily['SOFA_d{}'.format(day_num+1)]+df_max_daily['SOFA_d{}'.format(day_num)])/2

c_list = df_max_daily.columns
temp = df_max_daily[c_list[c_list.str.startswith('SOFA_d')]]
df_max_daily['SOFA_max'] =temp.max(axis=1)
df_max_daily['SOFA_min'] =temp.min(axis=1)
df_max_daily['SOFA_range'] = df_max_daily['SOFA_max']-df_max_daily['SOFA_min']
df_max_daily['SOFA_max_day'] =temp.idxmax(axis=1,skipna=True).str.slice(start=6)
df_max_daily['SOFA_min_day'] =temp.idxmin(axis=1,skipna=True).str.slice(start=6)
df_max_daily['SOFA_long_stay'] = ~pd.isnull(df_max_daily['SOFA_d8'])

df_max_daily['TotalAUC'] = df_max_daily[c_list[c_list.str.contains('Area'.format(day_num))]].sum(axis=1, skipna=True)

#coerces races into smaller list of categories
race_map =pd.read_csv('race_map.csv',header=None,index_col=0)
df_max_daily['race'] = df_max_daily['race'].map(race_map.to_dict()[1])

df_max_daily.to_csv("sofa_parameters.csv", index=False)

In [ ]:
# continue to analyze_results to run analysis